In [1]:
%%bash
wget -qnc https://raw.githubusercontent.com/gjoni/trDesign/beta/02-GD/models.py
wget -qnc https://raw.githubusercontent.com/gjoni/trDesign/beta/02-GD/utils.py
wget -qnc https://files.ipd.uw.edu/krypton/TrRosetta/design/to_pdb.py
wget -qnc https://files.ipd.uw.edu/krypton/TrRosetta/models.zip
wget -qnc https://files.ipd.uw.edu/krypton/TrRosetta/bkgr_models.zip
unzip -qqo models.zip
unzip -qqo bkgr_models.zip

In [2]:
%%bash
wget -qnc https://files.ipd.uw.edu/krypton/TrRosetta/model_TrMRF_seqid_retrain_5blocks.npy

In [3]:
%%bash
wget -qnc https://files.ipd.uw.edu/krypton/for_gabe.zip
unzip -qqo for_gabe.zip

In [4]:
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
os.chdir('/home/jupyter-yehlin/DMSV2/models')


import numpy as numpy
from scipy.special import softmax
from utils import *
from models import *
from to_pdb import *
from Bio.PDB.DSSP import dssp_dict_from_pdb_file

In [5]:
## tensorflow-gpu 2.8.0

In [8]:
model = mk_design_model(add_pdb=True, 
                        add_TrMRF=True, 
                        add_TrROS=True,
                        msa_design=False)

mode: single sequence design
loading model: xaa
loading model: xab
loading model: xac
loading model: xad
loading model: xae
loading model: TrMRF
The loss function is composed of the following: ['TrMRF', 'pdb']
Layer: input_1
Layer: tf_op_layer_Shape
Layer: tf_op_layer_random_uniform/RandomUniform
Layer: tf_op_layer_random_uniform/mul
Layer: tf_op_layer_random_uniform
Layer: tf_op_layer_add
Layer: tf_op_layer_Log
Layer: tf_op_layer_Neg
Layer: tf_op_layer_add_1
Layer: tf_op_layer_Log_1
Layer: tf_op_layer_Neg_1
Layer: input_6
Layer: input_4
Layer: tf_op_layer_add_2
Layer: tf_op_layer_strided_slice_2
Layer: tf_op_layer_strided_slice
Layer: tf_op_layer_truediv
Layer: tf_op_layer_Softmax_1
Layer: tf_op_layer_cond/pred_id
Layer: tf_op_layer_Softmax
Layer: tf_op_layer_cond/Switch_2
Layer: tf_op_layer_cond/Switch_1
Layer: tf_op_layer_cond/Merge
Layer: input_5
Layer: tf_op_layer_Shape_1
Layer: tf_op_layer_strided_slice_1
Layer: tf_op_layer_ArgMax
Layer: tf_op_layer_strided_slice_5
Layer: tf_op_l

In [9]:
def chk_id(a,b):
  return (np.array(list(a)) == np.array(list(b))).mean()

def set_ss(pdb):
  out = !~/bin/stride {pdb} | grep ^ASG
  C,H,E = 0,0,0
  for line in out:
    ss = line.split()[5].replace("T","C")
    if ss == "C": C += 1
    if ss == "H": H += 1
    if ss == "E": E += 1
  return C,H,E

In [ ]:
pdb_file = f"top_pdb/r9_900.pdb"
inputs = prep_input(pdb_file)
_feat = inputs["feat"]
_seq = np.eye(20)[AA_to_N(inputs["seq"])]

In [16]:
design = model.design(inputs={"pdb":_feat[None],
                            "I":_seq[None,None]}, 
                    opt_iter=100,
                    hard=False, hard_switch=[50],
                    num=100,
                    return_traj=False, verbose=True, seqid=1.0)
                    # rm_aa=[4])

0
1
2
3
4
FINAL loss:{'TrMRF':2.0011137,'pdb':1.5554913}


In [14]:
seqs = design["I"][0].argmax(-1)
wt_seq = _seq.argmax(-1)
idents = (wt_seq == seqs).mean(-1)

In [1]:
for pdb in os.listdir('top_pdb'):
    pdb_file = f"top_pdb/{pdb}"

    # get pdb features
    inputs = prep_input(pdb_file)
    _feat = inputs["feat"]
    _seq = np.eye(20)[AA_to_N(inputs["seq"])]

    design = model.design(inputs={"pdb":_feat[None],
                                "I":_seq[None,None]}, 
                        opt_iter=50,
                        hard=False, hard_switch=[50],
                        num=100,
                        return_traj=False, verbose=False, seqid=1.0,
                        rm_aa=[4])

    seqs = design["I"][0].argmax(-1)
    wt_seq = _seq.argmax(-1)
    idents = (wt_seq == seqs).mean(-1)

    NEW_seq = N_to_AA(seqs[idents.argmax()])[0]
    print(pdb, NEW_seq, round(chk_id(inputs["seq"][0],NEW_seq),3))
    with open("results.txt", "a") as file:
        file.write(f"{pdb} {NEW_seq} {round(chk_id(inputs['seq'][0],NEW_seq),3)}\n")

In [ ]:
# for pdb,(ln,sco) in scos.items():
#   pdb_file = f"top_pdb/{pdb}"

#   # get pdb features
#   inputs = prep_input(pdb_file)
#   _feat = inputs["feat"]
#   _seq = np.eye(20)[AA_to_N(inputs["seq"])]

#   design = model.design(inputs={"pdb":_feat[None],
#                                 "I":_seq[None,None]}, 
#                         opt_iter=100,
#                         hard=False, hard_switch=[50],
#                         num=100,
#                         return_traj=False, verbose=False, seqid=1.0,
#                         rm_aa=[4])
  
#   seqs = design["I"][0].argmax(-1)
#   wt_seq = _seq.argmax(-1)
#   idents = (wt_seq == seqs).mean(-1)

#   NEW_seq = N_to_AA(seqs[idents.argmax()])[0]
#   print(pdb, NEW_seq, round(chk_id(inputs["seq"][0],NEW_seq),3))
#   with open("results.txt", "a") as file:
#     file.write(f"{pdb} {NEW_seq} {round(chk_id(inputs['seq'][0],NEW_seq),3)}\n")